In [1]:
from models import *
from config import *

langs = LanguageModel(config)

output_seqs = langs(["what is that?"])

for s in output_seqs:
    print(s.shape)

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 8] nodename nor servname provided, or not
[nltk_data]     known>
